In [ ]:
import os

training_images_dir = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Images/images"

def count_images(directory):
    total_dirs = 0
    total_images = 0


    for entry in os.listdir(directory):
        path = os.path.join(directory, entry)
        if os.path.isdir(path):
            total_dirs += 1

            total_images += len([file for file in os.listdir(path) if os.path.isfile(os.path.join(path, file))])

    return total_dirs, total_images

num_dirs, num_images = count_images(training_images_dir)

print(f"There are {num_dirs} subdirectories in total")
print(f"All subdirectories together contain {num_images} images in total")


In [ ]:
import os

training_images_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/images_set_filter/training_images"

def count_images(directory):
    total_dirs = 0
    total_images = 0


    for entry in os.listdir(directory):
        path = os.path.join(directory, entry)
        if os.path.isdir(path):
            total_dirs += 1

            total_images += len([file for file in os.listdir(path) if os.path.isfile(os.path.join(path, file))])

    return total_dirs, total_images

num_dirs, num_images = count_images(training_images_dir)

print(f"There are {num_dirs} subdirectories in total")
print(f"All subdirectories together contain {num_images} images in total")


In [ ]:
import os

test_images_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/images_set_filter/test_images"

def count_images(directory):
    total_dirs = 0
    total_images = 0


    for entry in os.listdir(directory):
        path = os.path.join(directory, entry)
        if os.path.isdir(path):
            total_dirs += 1

            total_images += len([file for file in os.listdir(path) if os.path.isfile(os.path.join(path, file))])

    return total_dirs, total_images

num_dirs, num_images = count_images(test_images_dir)

print(f"There are {num_dirs} subdirectories in total")
print(f"All subdirectories together contain {num_images} images in total")


In [3]:

import pickle

path = "/mnt/dataset0/ldy/datasets/THINGS_MEG/Preprocessed/sub-02/preprocessed_meg_training.pkl"


with open(path, 'rb') as file:
    data = pickle.load(file)



meg_data = data['meg_data']
# ch_names = data['ch_names']
# times = data['times']
# meg_data.shape
meg_data.shape

(19848, 271, 201)

In [2]:

import pickle

path = "/mnt/dataset0/ldy/datasets/THINGS_MEG/Preprocessed/sub-02/preprocessed_meg_test.pkl"


with open(path, 'rb') as file:
    data = pickle.load(file)



meg_data = data['meg_data']
# ch_names = data['ch_names']
# times = data['times']
meg_data.shape


(200, 271, 201)

In [ ]:
import torch

meg_test_features = torch.load("/mnt/dataset0/ldy/Workspace/Thanos/data_preparing/MEG_ViT-H-14_features_test.pt")
meg_test_features['img_features'].shape





In [ ]:
meg_train_features = torch.load("/mnt/dataset0/ldy/Workspace/Thanos/data_preparing/MEG_ViT-H-14_features_train.pt")
meg_train_features['img_features'].shape


In [3]:
import os
import pickle
import numpy as np
import mne
from tqdm import tqdm
from collections import defaultdict
from random import choice
import pandas as pd

# 目录设置
base_fif_dir = "/mnt/dataset0/ldy/datasets/meg_dataset/original_preprocessed/preprocessed"
source_image_dir = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Images/images"
meg_image_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/images_set_12"
output_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/Preprocessed"
subjects = [
    # ('sub-01', 'preprocessed_P1-epo.fif'),
    ('sub-02', 'preprocessed_P2-epo.fif'),
    # ('sub-03', 'preprocessed_P3-epo.fif'),
    # ('sub-04', 'preprocessed_P4-epo.fif'),
]


In [4]:
def save_data(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)



def load_and_crop_epochs(fif_file):
    epochs = mne.read_epochs(fif_file, preload=True)
    epochs.crop(tmin=0, tmax=1.0)
    data_array = epochs.get_data()
    event_ids = epochs.events[:, 2]
    
    # 获取非 999999 的索引
    valid_indices = np.where(event_ids != 999999)[0]
    
    # 使用 valid_indices 筛选 data_array 和 event_ids
    data_array = data_array[valid_indices]
    event_ids = event_ids[valid_indices]

    print("event_ids", event_ids.shape)
    
    print("data_array", data_array.shape)
        
    # 获取唯一的 event_ids 的索引
    unique_indices = np.unique(event_ids, return_index=True)[1]
    
    # 根据 unique_indices 筛选 data_array 和 event_ids
    data_array = data_array[unique_indices]
    event_ids = event_ids[unique_indices]
    
    print("event_ids", event_ids.shape)
    print("unique_indices", unique_indices.shape)
    print("data_array", data_array.shape)
    
    return data_array, event_ids, epochs.ch_names, epochs.times

In [ ]:
# for subject_id, fif_filename in subjects:
fif_file = os.path.join(base_fif_dir, 'preprocessed_P2-epo.fif')
data_array, event_ids, ch_names, times = load_and_crop_epochs(fif_file)


In [ ]:
data_array.shape  #1654×12+200×12 

In [ ]:
event_ids[:50]

In [ ]:
event_ids

In [ ]:
len(event_ids)

In [ ]:
import os
import shutil
from pathlib import Path
import pandas as pd
import mne
import numpy as np

# 目录设置
csv_img_file_path = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Metadata/Image-specific/image_paths.csv"
base_fif_dir = "/mnt/dataset0/ldy/datasets/meg_dataset/original_preprocessed/preprocessed"
source_image_dir = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Images/"
training_images_dir = Path("/mnt/dataset0/ldy/datasets/THINGS_MEG/images_set/training_images")
test_images_dir = Path("/mnt/dataset0/ldy/datasets/THINGS_MEG/images_set/test_images")

train_dir = Path("/mnt/dataset0/ldy/datasets/THINGS_EEG/images_set/training_images")
test_dir = Path("/mnt/dataset0/ldy/datasets/THINGS_EEG/images_set/test_images")

subjects = [
    # ('sub-01', 'preprocessed_P1-epo.fif'),
    ('sub-02', 'preprocessed_P2-epo.fif'),
    # ('sub-03', 'preprocessed_P3-epo.fif'),
    # ('sub-04', 'preprocessed_P4-epo.fif'),
]

# 提取类别编号和真实标签
def get_label_dict(image_set_dir):
    label_dict = {}
    for subdir in image_set_dir.iterdir():
        if subdir.is_dir():
            folder_name = subdir.name
            index, label = folder_name.split("_", 1)  # 使用下划线分割编号和真实标签
            label_dict[label] = folder_name  # 保存真实标签和完整文件夹名
    return label_dict

# 获取训练和测试标签字典
train_label_dict = get_label_dict(train_dir)
test_label_dict = get_label_dict(test_dir)

# 读取 CSV 文件并创建图像路径映射
def load_image_paths(csv_path):
    df = pd.read_csv(csv_path, header=None)
    image_paths = df[0].tolist()  # 提取第一列作为图像路径
    return image_paths

# 加载图像路径
image_paths = load_image_paths(csv_img_file_path)
print("image_paths:", image_paths)

# 读取 FIF 文件并筛选数据
def load_and_crop_epochs(fif_file):
    epochs = mne.read_epochs(fif_file, preload=True)
    epochs.crop(tmin=0, tmax=1.0)
    data_array = epochs.get_data()
    event_ids = epochs.events[:, 2]
    
    # 获取非 999999 的索引
    valid_indices = np.where(event_ids != 999999)[0]
    data_array = data_array[valid_indices]
    event_ids = event_ids[valid_indices]
    
    # 获取唯一的 event_ids 的索引
    unique_indices = np.unique(event_ids, return_index=True)[1]
    data_array = data_array[unique_indices]
    event_ids = event_ids[unique_indices]
    
    return data_array, event_ids

# 复制图像到训练集和测试集
def copy_images(subject_id, event_ids):
    for event_id in event_ids:
        # 获取图片的相对路径
        image_path = image_paths[event_id - 1]
        full_image_path = os.path.join(source_image_dir, image_path)
        
        # 确定目标路径（训练或测试）
        image_label = image_path.split("/")[1]  # 假设标签是文件夹名称
        if image_label in train_label_dict:
            target_dir = training_images_dir / train_label_dict[image_label]
        elif image_label in test_label_dict:
            target_dir = test_images_dir / test_label_dict[image_label]
        else:
            print("对不上")

        target_dir.mkdir(parents=True, exist_ok=True)
        
        # 拷贝图片
        target_image_path = target_dir / os.path.basename(full_image_path)
        shutil.copy(full_image_path, target_image_path)

# 执行流程
for subject_id, fif_filename in subjects:
    fif_file = os.path.join(base_fif_dir, fif_filename)
    _, event_ids = load_and_crop_epochs(fif_file)
    copy_images(subject_id, event_ids)


In [ ]:
import os
import pickle
import numpy as np
import mne
from tqdm import tqdm
from collections import defaultdict
from random import choice
import pandas as pd


def save_data(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

def load_and_crop_epochs(fif_file):
    epochs = mne.read_epochs(fif_file, preload=True)
    epochs.crop(tmin=0, tmax=1.0)
    data_array = epochs.get_data()
    event_ids = epochs.events[:, 2]
    
    # 获取非 999999 的索引
    valid_indices = np.where(event_ids != 999999)[0]
    
    # 使用 valid_indices 筛选 data_array 和 event_ids
    data_array = data_array[valid_indices]
    event_ids = event_ids[valid_indices]

    print("event_ids", event_ids.shape)    
    print("data_array", data_array.shape)
        
    # 获取唯一的 event_ids 的索引
    unique_indices = np.unique(event_ids, return_index=True)[1]
    
    # 根据 unique_indices 筛选 data_array 和 event_ids
    data_array = data_array[unique_indices]
    event_ids = event_ids[unique_indices]
    
    print("event_ids", event_ids.shape)
    print("unique_indices", unique_indices.shape)
    print("data_array", data_array.shape)
    
    return data_array, event_ids, epochs.ch_names, epochs.times

def build_category_prefix_mapping(meg_image_dir):
    category_prefix_mapping = {}
    for split in ["training_images", "test_images"]:
        split_dir = os.path.join(meg_image_dir, split)
        if os.path.isdir(split_dir):
            for category_with_prefix in os.listdir(split_dir):
                prefix, category = category_with_prefix.split("_", 1)
                category_prefix_mapping[category] = category_with_prefix
    return category_prefix_mapping

def get_full_image_event_mapping(base_dir, category_prefix_mapping):
    event_id = 1
    full_image_event_mapping = {}
    categories = sorted(os.listdir(base_dir))
    for category in categories:
        category_path = os.path.join(base_dir, category)
        if os.path.isdir(category_path) and category in category_prefix_mapping:
            category_with_prefix = category_prefix_mapping[category]
            images = sorted(os.listdir(category_path))
            for img in images:
                full_image_event_mapping[event_id] = (category_with_prefix, img)
                event_id += 1
    return full_image_event_mapping


def find_unmatched_keys(event_ids, image_event_mapping):
    return [key for key in image_event_mapping.keys() if key not in event_ids]

def insert_unmatched_events(unmatched_keys, image_event_mapping, data_array, event_ids):
    filled_data = []
    filled_event_ids = []
    insert_indices = []

    for key in unmatched_keys:
        category_with_prefix, img = image_event_mapping[key]
        same_category_indices = [
            i for i, event_id in enumerate(event_ids)
            if event_id in image_event_mapping and image_event_mapping[event_id][0] == category_with_prefix
        ]
        
        if same_category_indices:
            sample_index = choice(same_category_indices)
            filled_data.append(data_array[sample_index][np.newaxis, :].copy())
            filled_event_ids.append(key)
            insert_indices.append(list(image_event_mapping.keys()).index(key))
            
        else:
            print(f"警告：无法为类别 {category_with_prefix} 填充数据")

    filled_data = np.vstack(filled_data) if filled_data else np.empty((0, data_array.shape[1], data_array.shape[2]))

    return filled_data, filled_event_ids, insert_indices



# 目录设置
csv_img_file_path = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Metadata/Image-specific/image_paths.csv"
base_fif_dir = "/mnt/dataset0/ldy/datasets/meg_dataset/original_preprocessed/preprocessed"
source_image_dir = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Images/images"
meg_image_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/images_set"
output_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/Preprocessed"
subjects = [
    # ('sub-01', 'preprocessed_P1-epo.fif'),
    ('sub-02', 'preprocessed_P2-epo.fif'),
    # ('sub-03', 'preprocessed_P3-epo.fif'),
    # ('sub-04', 'preprocessed_P4-epo.fif'),
]

category_prefix_mapping = build_category_prefix_mapping(meg_image_dir)
full_image_event_mapping = get_full_image_event_mapping(source_image_dir, category_prefix_mapping)

train_images_expected = set()
test_images_expected = set()

for category_with_prefix in category_prefix_mapping.values():
    train_dir = os.path.join(meg_image_dir, "training_images", category_with_prefix)
    test_dir = os.path.join(meg_image_dir, "test_images", category_with_prefix)

    if os.path.exists(train_dir):
        train_images_expected.update(
            [f"{category_with_prefix}/{img}" for img in sorted(os.listdir(train_dir))]
        )
    if os.path.exists(test_dir):
        test_images_expected.update(
            [f"{category_with_prefix}/{img}" for img in sorted(os.listdir(test_dir))]
        )

image_event_mapping_train = {}
image_event_mapping_test = {}

for event_id, (category_with_prefix, img) in full_image_event_mapping.items():
    train_path = os.path.join(meg_image_dir, "training_images", category_with_prefix, img)
    test_path = os.path.join(meg_image_dir, "test_images", category_with_prefix, img)
    if os.path.exists(train_path):
        image_event_mapping_train[event_id] = (category_with_prefix, img)
    elif os.path.exists(test_path):
        image_event_mapping_test[event_id] = (category_with_prefix, img)

# image_event_mapping_train

# 将字典转换为 DataFrame，便于查看
# image_event_mapping_train_df = pd.DataFrame.from_dict(image_event_mapping_train, orient='index', columns=['Category_Prefix', 'Image'])

# # 保存为 CSV 文件
# train_file_path = "image_event_mapping_train.csv"
# image_event_mapping_train_df.to_csv(train_file_path, index_label="Event_ID")

# print(f"文件已保存到: {train_file_path}")



# image_event_mapping_test
# # 将字典转换为 DataFrame，便于查看
# image_event_mapping_test_df = pd.DataFrame.from_dict(image_event_mapping_test, orient='index', columns=['Category_Prefix', 'Image'])

# # 保存为 CSV 文件
# file_path = "image_event_mapping_test.csv"
# image_event_mapping_test_df.to_csv(file_path, index_label="Event_ID")

# print(f"文件已保存到: {file_path}")

for subject_id, fif_filename in subjects:
    fif_file = os.path.join(base_fif_dir, fif_filename)
    data_array, event_ids, ch_names, times = load_and_crop_epochs(fif_file)
    
    train_array = []
    test_array = []
    
    for i, event_id in enumerate(event_ids):
        if event_id in image_event_mapping_train:
            train_array.append(data_array[i])
        elif event_id in image_event_mapping_test:
            test_array.append(data_array[i])
        else:
            print("event_id 多余", event_id)
    train_array = np.array(train_array)
    test_array = np.array(test_array)
    
    subject_output_dir = os.path.join(output_dir, subject_id)
    os.makedirs(subject_output_dir, exist_ok=True)
    
    unmatched_train_keys = find_unmatched_keys(event_ids, image_event_mapping_train)
    unmatched_test_keys = find_unmatched_keys(event_ids, image_event_mapping_test)
    
    # filled_train_data, filled_train_ids, train_insert_indices = insert_unmatched_events(
    #     unmatched_train_keys, image_event_mapping_train, data_array, event_ids)
    # filled_test_data, filled_test_ids, test_insert_indices = insert_unmatched_events(
    #     unmatched_test_keys, image_event_mapping_test, data_array, event_ids)
    
    # Sort indices in descending order to avoid index shift during insertion
    # train_insert_indices = sorted(train_insert_indices)
    # test_insert_indices = sorted(test_insert_indices)

    # Insert filled data from the end of the array, ensuring indices stay consistent
    # if filled_train_data.size > 0:
    #     for data, idx in zip(filled_train_data, train_insert_indices):
    #         print("insert train idx", idx)
    #         train_array = np.insert(train_array, idx, data, axis=0)

    # if filled_test_data.size > 0:
    #     for data, idx in zip(filled_test_data, test_insert_indices):
    #         print("insert test idx", idx)
    #         test_array = np.insert(test_array, idx, data, axis=0)

    save_data({'meg_data': train_array, 'ch_names': ch_names, 'times': times},
              os.path.join(subject_output_dir, "preprocessed_meg_training.pkl"))
    save_data({'meg_data': test_array, 'ch_names': ch_names, 'times': times},
              os.path.join(subject_output_dir, "preprocessed_meg_test.pkl"))

    print(f"\nUnmatched keys in training set for {subject_id}:")
    print(unmatched_train_keys)
    
    print(f"\nUnmatched keys in test set for {subject_id}:")
    print(unmatched_test_keys)

print("处理完成！")


In [ ]:
import os
import pickle
import numpy as np
import mne
from collections import defaultdict
import pandas as pd

def save_data(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

def load_and_crop_epochs(fif_file):
    epochs = mne.read_epochs(fif_file, preload=True)
    epochs.crop(tmin=0, tmax=1.0)
    data_array = epochs.get_data()
    event_ids = epochs.events[:, 2]
    
    # 获取非 999999 的索引
    valid_indices = np.where(event_ids != 999999)[0]
    data_array = data_array[valid_indices]
    event_ids = event_ids[valid_indices]
    
    # 获取唯一的 event_ids 的索引
    unique_indices = np.unique(event_ids, return_index=True)[1]
    data_array = data_array[unique_indices]
    event_ids = event_ids[unique_indices]
    
    return data_array, event_ids, epochs.ch_names, epochs.times

def build_category_prefix_mapping(meg_image_dir):
    category_prefix_mapping = {}
    for split in ["training_images", "test_images"]:
        split_dir = os.path.join(meg_image_dir, split)
        if os.path.isdir(split_dir):
            for category_with_prefix in os.listdir(split_dir):
                prefix, category = category_with_prefix.split("_", 1)
                category_prefix_mapping[category] = category_with_prefix
    return category_prefix_mapping

def get_full_image_event_mapping(csv_img_file_path, category_prefix_mapping):
    full_image_event_mapping = {}
    
    # 使用 CSV 文件中图像路径顺序创建 event_id 映射
    df = pd.read_csv(csv_img_file_path, header=None)
    for event_id, row in enumerate(df[0], start=1):
        image_path = row
        category = image_path.split("/")[1]  # 假设类别是文件夹名称
        if category in category_prefix_mapping:
            category_with_prefix = category_prefix_mapping[category]
            img = os.path.basename(image_path)
            full_image_event_mapping[event_id] = (category_with_prefix, img)
            
    return full_image_event_mapping

def find_unmatched_keys(event_ids, image_event_mapping):
    return [key for key in image_event_mapping.keys() if key not in event_ids]

def insert_unmatched_events(unmatched_keys, image_event_mapping, data_array, event_ids):
    filled_data = []
    filled_event_ids = []
    insert_indices = []

    for key in unmatched_keys:
        category_with_prefix, img = image_event_mapping[key]
        same_category_indices = [
            i for i, event_id in enumerate(event_ids)
            if event_id in image_event_mapping and image_event_mapping[event_id][0] == category_with_prefix
        ]
        
        if same_category_indices:
            sample_index = choice(same_category_indices)
            filled_data.append(data_array[sample_index][np.newaxis, :].copy())
            filled_event_ids.append(key)
            insert_indices.append(list(image_event_mapping.keys()).index(key))
            
        else:
            print(f"警告：无法为类别 {category_with_prefix} 填充数据")

    filled_data = np.vstack(filled_data) if filled_data else np.empty((0, data_array.shape[1], data_array.shape[2]))

    return filled_data, filled_event_ids, insert_indices

# 目录设置
csv_img_file_path = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Metadata/Image-specific/image_paths.csv"
base_fif_dir = "/mnt/dataset0/ldy/datasets/meg_dataset/original_preprocessed/preprocessed"
source_image_dir = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Images/images"
meg_image_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/images_set"
output_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/Preprocessed"
subjects = [
    ('sub-02', 'preprocessed_P2-epo.fif'),
]

category_prefix_mapping = build_category_prefix_mapping(meg_image_dir)
full_image_event_mapping = get_full_image_event_mapping(csv_img_file_path, category_prefix_mapping)

image_event_mapping_train = {}
image_event_mapping_test = {}

for event_id, (category_with_prefix, img) in full_image_event_mapping.items():
    train_path = os.path.join(meg_image_dir, "training_images", category_with_prefix, img)
    test_path = os.path.join(meg_image_dir, "test_images", category_with_prefix, img)
    if os.path.exists(train_path):
        image_event_mapping_train[event_id] = (category_with_prefix, img)
    elif os.path.exists(test_path):
        image_event_mapping_test[event_id] = (category_with_prefix, img)

for subject_id, fif_filename in subjects:
    fif_file = os.path.join(base_fif_dir, fif_filename)
    data_array, event_ids, ch_names, times = load_and_crop_epochs(fif_file)
    
    train_array = []
    test_array = []
    
    for i, event_id in enumerate(event_ids):
        if event_id in image_event_mapping_train:
            train_array.append(data_array[i])
        elif event_id in image_event_mapping_test:
            test_array.append(data_array[i])
        else:
            print("event_id 多余", event_id)
    train_array = np.array(train_array)
    test_array = np.array(test_array)
    
    subject_output_dir = os.path.join(output_dir, subject_id)
    os.makedirs(subject_output_dir, exist_ok=True)
    
    unmatched_train_keys = find_unmatched_keys(event_ids, image_event_mapping_train)
    unmatched_test_keys = find_unmatched_keys(event_ids, image_event_mapping_test)
    
    save_data({'meg_data': train_array, 'ch_names': ch_names, 'times': times},
              os.path.join(subject_output_dir, "preprocessed_meg_training.pkl"))
    save_data({'meg_data': test_array, 'ch_names': ch_names, 'times': times},
              os.path.join(subject_output_dir, "preprocessed_meg_test.pkl"))

    print(f"\nUnmatched keys in training set for {subject_id}:")
    print(unmatched_train_keys)
    
    print(f"\nUnmatched keys in test set for {subject_id}:")
    print(unmatched_test_keys)

print("处理完成！")


In [4]:
import os
import pickle
import numpy as np
import mne
import pandas as pd
from collections import defaultdict
import shutil

def save_data(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

def load_and_crop_epochs(fif_file):
    epochs = mne.read_epochs(fif_file, preload=True)
    epochs.crop(tmin=0, tmax=1.0)
    data_array = epochs.get_data()
    event_ids = epochs.events[:, 2]
    
    # 获取非 999999 的索引
    valid_indices = np.where(event_ids != 999999)[0]
    data_array = data_array[valid_indices]
    event_ids = event_ids[valid_indices]
    
    # 获取唯一的 event_ids 的索引
    unique_indices = np.unique(event_ids, return_index=True)[1]
    data_array = data_array[unique_indices]
    event_ids = event_ids[unique_indices]
    
    return data_array, event_ids, epochs.ch_names, epochs.times

def build_category_prefix_mapping(meg_image_dir):
    category_prefix_mapping = {}
    for split in ["training_images", "test_images"]:
        split_dir = os.path.join(meg_image_dir, split)
        if os.path.isdir(split_dir):
            for category_with_prefix in os.listdir(split_dir):
                prefix, category = category_with_prefix.split("_", 1)
                category_prefix_mapping[category] = category_with_prefix
    return category_prefix_mapping

def get_full_image_event_mapping(csv_img_file_path, category_prefix_mapping):
    full_image_event_mapping = {}
    
    # 使用 CSV 文件中图像路径顺序创建 event_id 映射
    df = pd.read_csv(csv_img_file_path, header=None)
    for event_id, row in enumerate(df[0], start=1):
        image_path = row
        category = image_path.split("/")[1]  # 假设类别是文件夹名称
        if category in category_prefix_mapping:
            category_with_prefix = category_prefix_mapping[category]
            img = os.path.basename(image_path)
            full_image_event_mapping[event_id] = (category_with_prefix, img)
            
    return full_image_event_mapping

def filter_and_copy_images(image_event_mapping, data_array, event_ids, output_array, output_dir, category_limit=12):
    image_count = defaultdict(int)  # 用于记录每个类别的图像数量
    incomplete_categories = []  # 用于记录不满足12张图像的类别
    filtered_event_ids = []  # 用于存储过滤后的 event_id
    
    # 新建输出目录
    os.makedirs(output_dir, exist_ok=True)
    
    for i, event_id in enumerate(event_ids):
        if event_id not in image_event_mapping:
            continue
        category_with_prefix, img = image_event_mapping[event_id]
        
        # 检查每个类别的图像数量是否已达到限制
        if image_count[category_with_prefix] < category_limit:
            output_array.append(data_array[i])  # 保留该样本
            filtered_event_ids.append(event_id)  # 将符合条件的 event_id 添加到过滤列表
            image_count[category_with_prefix] += 1
            
            # 提取不带前缀的类别名
            # category = category_with_prefix.split("_", 1)[1]  # 提取不带前缀的类别名
            
            # 构建源路径（使用不带前缀的类别名）和目标路径
            # source_path = os.path.join(source_image_dir, "images", category, img)  # 使用不带前缀的类别名
            # category_output_dir = os.path.join(output_dir, category_with_prefix)  # 目标路径使用带前缀的类别名
            # os.makedirs(category_output_dir, exist_ok=True)
            # target_path = os.path.join(category_output_dir, img)
            
            # 拷贝文件
            # shutil.copyfile(source_path, target_path)
    
    # 检查是否存在不满足12张图像的类别
    for category, count in image_count.items():
        if count < category_limit:
            incomplete_categories.append((category, count))
    
    return incomplete_categories, filtered_event_ids

# 目录设置
csv_img_file_path = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Metadata/Image-specific/image_paths.csv"
base_fif_dir = "/mnt/dataset0/ldy/datasets/meg_dataset/original_preprocessed/preprocessed"
source_image_dir = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Images/"
meg_image_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/images_set"
output_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/Preprocessed"
filtered_image_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/images_set_filter"
subjects = [
    ('sub-02', 'preprocessed_P2-epo.fif'),
]

category_prefix_mapping = build_category_prefix_mapping(meg_image_dir)
full_image_event_mapping = get_full_image_event_mapping(csv_img_file_path, category_prefix_mapping)

image_event_mapping_train = {}
image_event_mapping_test = {}

for event_id, (category_with_prefix, img) in full_image_event_mapping.items():
    train_path = os.path.join(meg_image_dir, "training_images", category_with_prefix, img)
    test_path = os.path.join(meg_image_dir, "test_images", category_with_prefix, img)
    if os.path.exists(train_path):
        image_event_mapping_train[event_id] = (category_with_prefix, img)
    elif os.path.exists(test_path):
        image_event_mapping_test[event_id] = (category_with_prefix, img)

# 主处理代码
filtered_train_event_ids = []
filtered_test_event_ids = []

for subject_id, fif_filename in subjects:
    fif_file = os.path.join(base_fif_dir, fif_filename)
    data_array, event_ids, ch_names, times = load_and_crop_epochs(fif_file)
    print("event_ids", event_ids)
    train_array = []
    test_array = []
    
    # 过滤训练集和测试集，保留每个类别的前12张图像并获取过滤后的 event_ids
    incomplete_train_categories, filtered_train_event_ids = filter_and_copy_images(
        image_event_mapping_train, data_array, event_ids, train_array, os.path.join(filtered_image_dir, "training_images"))
    incomplete_test_categories, filtered_test_event_ids = filter_and_copy_images(
        image_event_mapping_test, data_array, event_ids, test_array, os.path.join(filtered_image_dir, "test_images"))

    # 将结果转换为 numpy 数组
    train_array = np.array(train_array)
    test_array = np.array(test_array)
    
    subject_output_dir = os.path.join(output_dir, subject_id)
    os.makedirs(subject_output_dir, exist_ok=True)
    
    save_data({'meg_data': train_array, 'ch_names': ch_names, 'times': times},
              os.path.join(subject_output_dir, "preprocessed_meg_training.pkl"))
    save_data({'meg_data': test_array, 'ch_names': ch_names, 'times': times},
              os.path.join(subject_output_dir, "preprocessed_meg_test.pkl"))

    # 输出不满足12张图像的类别
    if incomplete_train_categories:
        print(f"\n训练集中不满足12张图像的类别（数量不足） for {subject_id}:")
        for category, count in incomplete_train_categories:
            print(f"  类别 '{category}': 仅 {count} 张图像")
    
    if incomplete_test_categories:
        print(f"\n测试集中不满足12张图像的类别（数量不足） for {subject_id}:")
        for category, count in incomplete_test_categories:
            print(f"  类别 '{category}': 仅 {count} 张图像")

# 处理完成后，直接使用 filtered_train_event_ids 和 filtered_test_event_ids
print("处理完成！")
print("过滤后的训练集的 event_ids:", filtered_train_event_ids)
print("过滤后的测试集的 event_ids:", filtered_test_event_ids)

# 使用epochs整体

In [1]:
import os
import pickle
import numpy as np
import mne
import pandas as pd
from collections import defaultdict
import shutil


def load_and_crop_epochs(fif_file):
    epochs = mne.read_epochs(fif_file, preload=True)
    epochs.crop(tmin=0, tmax=1.0)
    
    sorted_indices = np.argsort(epochs.events[:, 2])
    epochs = epochs[sorted_indices]
    
    filtered_epochs = epochs[epochs.events[:, 2] != 999999]
    
    
    # 根据事件进行去重处理
    unique_events, unique_indices = np.unique(filtered_epochs.events[:, 2], return_index=True)
    unique_filtered_epochs = filtered_epochs[unique_indices]    
    
    return unique_filtered_epochs


# 读取 meg 类别标签文件
csv_file_path = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Metadata/Concept-specific/image_concept_index.csv"
concept_df = pd.read_csv(csv_file_path, header=None, names=['Category_Label'])

subjects = [
    ('sub-02', 'preprocessed_P2-epo.fif'),
]
base_fif_dir = "/mnt/dataset0/ldy/datasets/meg_dataset/original_preprocessed/preprocessed"

for subject_id, fif_filename in subjects:
    fif_file = os.path.join(base_fif_dir, fif_filename)
    epochs = load_and_crop_epochs(fif_file)





Reading /mnt/dataset0/ldy/datasets/meg_dataset/original_preprocessed/preprocessed/preprocessed_P2-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...    1300.00 ms
        0 CTF compensation matrices available
Reading /mnt/dataset0/ldy/datasets/meg_dataset/original_preprocessed/preprocessed/preprocessed_P2-epo-1.fif ...
    Found the data of interest:
        t =    -100.00 ...    1300.00 ms
        0 CTF compensation matrices available
Reading /mnt/dataset0/ldy/datasets/meg_dataset/original_preprocessed/preprocessed/preprocessed_P2-epo-2.fif ...
    Found the data of interest:
        t =    -100.00 ...    1300.00 ms
        0 CTF compensation matrices available
Reading /mnt/dataset0/ldy/datasets/meg_dataset/original_preprocessed/preprocessed/preprocessed_P2-epo-3.fif ...
    Found the data of interest:
        t =    -100.00 ...    1300.00 ms
        0 CTF compensation matrices available
Adding metadata with 18 columns
27048 matching events found
No baseline correc

In [2]:
epochs.times.shape

(201,)

In [3]:
epochs.events[:, 2]

array([    1,     2,     3, ..., 26104, 26105, 26106], dtype=int32)

In [4]:
len(epochs)

22448

In [5]:
epochs.events[2, 2]

3

In [6]:
def save_data(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

In [7]:
import os
import pandas as pd
import os
import pickle
import numpy as np
import mne
import pandas as pd
from collections import defaultdict
import shutil

# 读取 meg 类别标签文件
csv_file_path = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Metadata/Concept-specific/image_concept_index.csv"
concept_df = pd.read_csv(csv_file_path, header=None, names=['Category_Label'])
# 检查行号是否与 event_id 一致
# 如果每行对应的行号就是 event_id，则行号即是 event_id
concept_df['Event_ID'] = concept_df.index + 1  # 添加 Event_ID 列


def count_samples_per_category(event_ids, concept_df):
    # 筛选出在过滤后的 event_ids 中的类别标签
    filtered_concept_df = concept_df[concept_df['Event_ID'].isin(event_ids)]

    # 统计每个类别在过滤后的数据中的样本数量
    category_counts = filtered_concept_df['Category_Label'].value_counts().sort_index()

    # 将统计结果转为字典并返回
    return category_counts.to_dict()

# 调用函数进行统计
category_counts = count_samples_per_category(epochs.events[:, 2], concept_df)

# 输出结果
print("每个类别在过滤后的 MEG 数据中样本数量:")
for category, count in category_counts.items():
    print(f"类别 {category}: {count} 个样本")

每个类别在过滤后的 MEG 数据中样本数量:
类别 1: 12 个样本
类别 2: 12 个样本
类别 3: 12 个样本
类别 4: 12 个样本
类别 5: 12 个样本
类别 6: 12 个样本
类别 7: 12 个样本
类别 8: 12 个样本
类别 9: 12 个样本
类别 10: 13 个样本
类别 11: 12 个样本
类别 12: 12 个样本
类别 13: 13 个样本
类别 14: 12 个样本
类别 15: 12 个样本
类别 16: 12 个样本
类别 17: 13 个样本
类别 18: 13 个样本
类别 19: 12 个样本
类别 20: 12 个样本
类别 21: 12 个样本
类别 22: 12 个样本
类别 23: 12 个样本
类别 24: 12 个样本
类别 25: 12 个样本
类别 26: 12 个样本
类别 27: 12 个样本
类别 28: 12 个样本
类别 29: 12 个样本
类别 30: 12 个样本
类别 31: 12 个样本
类别 32: 12 个样本
类别 33: 12 个样本
类别 34: 12 个样本
类别 35: 12 个样本
类别 36: 12 个样本
类别 37: 12 个样本
类别 38: 12 个样本
类别 39: 12 个样本
类别 40: 12 个样本
类别 41: 12 个样本
类别 42: 13 个样本
类别 43: 12 个样本
类别 44: 13 个样本
类别 45: 12 个样本
类别 46: 13 个样本
类别 47: 13 个样本
类别 48: 12 个样本
类别 49: 12 个样本
类别 50: 12 个样本
类别 51: 12 个样本
类别 52: 12 个样本
类别 53: 12 个样本
类别 54: 12 个样本
类别 55: 13 个样本
类别 56: 13 个样本
类别 57: 12 个样本
类别 58: 12 个样本
类别 59: 12 个样本
类别 60: 13 个样本
类别 61: 13 个样本
类别 62: 12 个样本
类别 63: 13 个样本
类别 64: 13 个样本
类别 65: 12 个样本
类别 66: 12 个样本
类别 67: 12 个样本
类别 68: 12 个样本
类别 69: 12 个样本
类别 70: 12 个样本
类别 71:

In [11]:
epochs.events[:, 2]

array([    1,     2,     3, ..., 26104, 26105, 26106], dtype=int32)

In [12]:
# 目录设置
csv_img_file_path = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Metadata/Image-specific/image_paths.csv"
base_fif_dir = "/mnt/dataset0/ldy/datasets/meg_dataset/original_preprocessed/preprocessed"
source_image_dir = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Images/"
# meg_image_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/images_set"
meg_image_dir = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/images_set"
output_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/Preprocessed"
filtered_image_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/images_set_filter"


def build_category_prefix_mapping(meg_image_dir):
    category_prefix_mapping = {}
    for split in ["training_images/images", "test_images/images"]:
        split_dir = os.path.join(meg_image_dir, split)
        if os.path.isdir(split_dir):
            for category_with_prefix in os.listdir(split_dir):
                prefix, category = category_with_prefix.split("_", 1)
                category_prefix_mapping[category] = category_with_prefix
    return category_prefix_mapping


def get_full_image_event_mapping(csv_img_file_path, category_prefix_mapping):
    full_image_event_mapping = {}
    
    # 使用 CSV 文件中图像路径顺序创建 event_id 映射
    df = pd.read_csv(csv_img_file_path, header=None)
    for event_id, row in enumerate(df[0], start=1):
        image_path = row
        
        category = image_path.split("/")[1]  # 假设类别是文件夹名称
        # print(image_path.split("/"))
        if category in category_prefix_mapping:
            category_with_prefix = category_prefix_mapping[category]
            img = os.path.basename(image_path)
            full_image_event_mapping[event_id] = (category_with_prefix, img)
            
    return full_image_event_mapping


def filter_and_copy_images(epochs, image_event_mapping_train, image_event_mapping_test, category_limit=12):
    image_count_test = defaultdict(int)
    image_count_train = defaultdict(int)
    test_indices = []
    train_indices = []
    incomplete_categories = []

    # 按 event_id 对 epochs 进行排序
    sorted_indices = np.argsort(epochs.events[:, 2])
    epochs = epochs[sorted_indices]

    # 新建输出目录
    os.makedirs(output_dir, exist_ok=True)

    # 仅遍历 epochs 一次，区分训练集和测试集
    for epoch_idx in range(len(epochs)):
        event_id = epochs.events[epoch_idx, 2]

        # 判断事件是否属于测试集
        if event_id in image_event_mapping_test:
            category_with_prefix, img = image_event_mapping_test[event_id]
            # 检查测试集每个类别的图像数量是否已达到限制
            if image_count_test[category_with_prefix] < category_limit:
                test_indices.append(epoch_idx)  # 记录测试集样本的索引
                image_count_test[category_with_prefix] += 1

        # 判断事件是否属于训练集
        elif event_id in image_event_mapping_train:
            category_with_prefix, img = image_event_mapping_train[event_id]
            # 检查训练集每个类别的图像数量是否已达到限制
            if image_count_train[category_with_prefix] < category_limit:
                train_indices.append(epoch_idx)  # 记录训练集样本的索引
                image_count_train[category_with_prefix] += 1

    # 根据索引提取过滤后的 train_epochs 和 test_epochs
    train_epochs = epochs[train_indices]
    test_epochs = epochs[test_indices]

    # 检查测试集和训练集中是否存在不满足12张图像的类别
    for category, count in image_count_test.items():
        if count < category_limit:
            incomplete_categories.append((category, count))
    for category, count in image_count_train.items():
        if count < category_limit:
            incomplete_categories.append((category, count))

    return train_epochs, test_epochs, incomplete_categories


In [14]:
# 初始化 category_prefix_mapping 和 image_event_mapping
category_prefix_mapping = build_category_prefix_mapping(meg_image_dir)
full_image_event_mapping = get_full_image_event_mapping(csv_img_file_path, category_prefix_mapping)

image_event_mapping_train = {}
image_event_mapping_test = {}

# 将事件分类到训练集和测试集
for event_id, (category_with_prefix, img) in full_image_event_mapping.items():
    train_path = os.path.join(meg_image_dir, "training_images/images", category_with_prefix, img)
    test_path = os.path.join(meg_image_dir, "test_images/images", category_with_prefix, img)
    # print(test_path)
    if os.path.exists(train_path):
        image_event_mapping_train[event_id] = (category_with_prefix, img)
    elif os.path.exists(test_path):
        image_event_mapping_test[event_id] = (category_with_prefix, img)

In [15]:
# 主处理代码
filtered_train_event_ids = []
filtered_test_event_ids = []

# for subject_id, fif_filename in subjects:
#     fif_file = os.path.join(base_fif_dir, fif_filename)
#     epochs = load_and_crop_epochs(fif_file)
    
# 使用修改后的函数来划分训练集和测试集
train_epochs, test_epochs, incomplete_categories = filter_and_copy_images(
    epochs, image_event_mapping_train, image_event_mapping_test, category_limit=12 )




In [16]:
incomplete_categories

[('00010_aircraft_carrier', 1),
 ('00013_alligator', 1),
 ('00017_altar', 1),
 ('00018_aluminum_foil', 1),
 ('00042_ashtray', 1),
 ('00044_avocado', 1),
 ('00046_axe', 1),
 ('00047_baby', 1),
 ('00055_bag', 1),
 ('00056_bagel', 1),
 ('00060_ball', 1),
 ('00061_balloon', 1),
 ('00063_bamboo', 1),
 ('00064_banana', 1),
 ('00086_bassinet', 1),
 ('00098_battery', 1),
 ('00100_beachball', 1),
 ('00103_bean', 1),
 ('00106_bear', 1),
 ('00108_beaver', 1),
 ('00109_bed', 1),
 ('00112_bee', 1),
 ('00114_beer', 1),
 ('00117_bell', 1),
 ('00121_bench', 1),
 ('00124_bike', 1),
 ('00141_blind', 1),
 ('00149_blowtorch', 1),
 ('00151_boa', 1),
 ('00155_boat', 1),
 ('00156_bobsled', 1),
 ('00183_boxer_shorts', 1),
 ('00187_brace', 1),
 ('00202_broccoli', 1),
 ('00205_brownie', 1),
 ('00216_bulldozer', 1),
 ('00217_bullet', 1),
 ('00230_butterfly', 1),
 ('00246_camel', 1),
 ('00253_candelabra', 1),
 ('00263_canoe', 1),
 ('00274_cardigan', 1),
 ('00282_cashew', 1),
 ('00287_caterpillar', 1),
 ('00313_ch

In [1]:
len(train_epochs)

NameError: name 'train_epochs' is not defined

In [18]:
test_epochs.events[:, 2][:50]

array([ 148,  191,  244,  264,  591,  619,  646,  663,  808,  828,  895,
        914,  944,  960, 1279, 1481, 1510, 1556, 1610, 1640, 1664, 1719,
       1748, 1794, 1858, 1899, 2159, 2286, 2312, 2378, 2393, 2783, 2839,
       3061, 3099, 3255, 3268, 3435, 3651, 3739, 3880, 4030, 4137, 4205,
       4562, 4603, 4654, 4851, 5161, 5226], dtype=int32)

In [19]:
times = train_epochs.times 
times

array([0.   , 0.005, 0.01 , 0.015, 0.02 , 0.025, 0.03 , 0.035, 0.04 ,
       0.045, 0.05 , 0.055, 0.06 , 0.065, 0.07 , 0.075, 0.08 , 0.085,
       0.09 , 0.095, 0.1  , 0.105, 0.11 , 0.115, 0.12 , 0.125, 0.13 ,
       0.135, 0.14 , 0.145, 0.15 , 0.155, 0.16 , 0.165, 0.17 , 0.175,
       0.18 , 0.185, 0.19 , 0.195, 0.2  , 0.205, 0.21 , 0.215, 0.22 ,
       0.225, 0.23 , 0.235, 0.24 , 0.245, 0.25 , 0.255, 0.26 , 0.265,
       0.27 , 0.275, 0.28 , 0.285, 0.29 , 0.295, 0.3  , 0.305, 0.31 ,
       0.315, 0.32 , 0.325, 0.33 , 0.335, 0.34 , 0.345, 0.35 , 0.355,
       0.36 , 0.365, 0.37 , 0.375, 0.38 , 0.385, 0.39 , 0.395, 0.4  ,
       0.405, 0.41 , 0.415, 0.42 , 0.425, 0.43 , 0.435, 0.44 , 0.445,
       0.45 , 0.455, 0.46 , 0.465, 0.47 , 0.475, 0.48 , 0.485, 0.49 ,
       0.495, 0.5  , 0.505, 0.51 , 0.515, 0.52 , 0.525, 0.53 , 0.535,
       0.54 , 0.545, 0.55 , 0.555, 0.56 , 0.565, 0.57 , 0.575, 0.58 ,
       0.585, 0.59 , 0.595, 0.6  , 0.605, 0.61 , 0.615, 0.62 , 0.625,
       0.63 , 0.635,

In [21]:
subject_output_dir

'/mnt/dataset0/ldy/datasets/THINGS_MEG/Preprocessed/sub-02'

In [20]:
# 设置输出路径
subject_output_dir = os.path.join(output_dir, subject_id)
os.makedirs(subject_output_dir, exist_ok=True)

# 从训练和测试 epochs 中提取数据
train_data = train_epochs.get_data()  # 提取数据
test_data = test_epochs.get_data()  # 提取数据
ch_names = train_epochs.ch_names  # 提取通道名称
times = train_epochs.times  # 提取时间点信息

# 使用 save_data 函数保存数据为 pkl 格式
save_data({'meg_data': train_data, 'ch_names': ch_names, 'times': times},
            os.path.join(subject_output_dir, "preprocessed_meg_training.pkl"))
save_data({'meg_data': test_data, 'ch_names': ch_names, 'times': times},
            os.path.join(subject_output_dir, "preprocessed_meg_test.pkl"))

# # 输出不满足12张图像的类别
# if incomplete_train_categories:
#     print(f"\n训练集中不满足12张图像的类别（数量不足） for {subject_id}:")
#     for category, count in incomplete_train_categories:
#         print(f"  类别 '{category}': 仅 {count} 张图像")

# if incomplete_test_categories:
#     print(f"\n测试集中不满足12张图像的类别（数量不足） for {subject_id}:")
#     for category, count in incomplete_test_categories:
#         print(f"  类别 '{category}': 仅 {count} 张图像")

print("处理完成！")

处理完成！


In [ ]:
import os
import pickle
import numpy as np
import mne
import pandas as pd
from collections import defaultdict
import shutil

def save_data(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

def load_and_crop_epochs(fif_file):
    epochs = mne.read_epochs(fif_file, preload=True)
    epochs.crop(tmin=0, tmax=1.0)
    
    sorted_indices = np.argsort(epochs.events[:, 2])
    epochs = epochs[sorted_indices]
    
    filtered_epochs = epochs[epochs.events[:, 2] != 999999]
    unique_event_ids, counts = np.unique(epochs.events[:, 2], return_counts=True)
    valid_epochs = filtered_epochs[np.isin(filtered_epochs.events[:, 2], unique_event_ids)]
    # # 获取非 999999 的索引
    # valid_indices = np.where(event_ids != 999999)[0]
    # data_array = data_array[valid_indices]
    # event_ids = event_ids[valid_indices]
    
    # # 获取唯一的 event_ids 的索引
    # unique_indices = np.unique(event_ids, return_index=True)[1]
    # data_array = data_array[unique_indices]
    # event_ids = event_ids[unique_indices]
    
    return valid_epochs

def build_category_prefix_mapping(meg_image_dir):
    category_prefix_mapping = {}
    for split in ["training_images", "test_images"]:
        split_dir = os.path.join(meg_image_dir, split)
        if os.path.isdir(split_dir):
            for category_with_prefix in os.listdir(split_dir):
                prefix, category = category_with_prefix.split("_", 1)
                category_prefix_mapping[category] = category_with_prefix
    return category_prefix_mapping

def get_full_image_event_mapping(csv_img_file_path, category_prefix_mapping):
    full_image_event_mapping = {}
    
    # 使用 CSV 文件中图像路径顺序创建 event_id 映射
    df = pd.read_csv(csv_img_file_path, header=None)
    for event_id, row in enumerate(df[0], start=1):
        image_path = row
        category = image_path.split("/")[1]  # 假设类别是文件夹名称
        if category in category_prefix_mapping:
            category_with_prefix = category_prefix_mapping[category]
            img = os.path.basename(image_path)
            full_image_event_mapping[event_id] = (category_with_prefix, img)
            
    return full_image_event_mapping

def filter_and_copy_images(image_event_mapping, data_array, event_ids, output_array, output_dir, category_limit=12):
    image_count = defaultdict(int)  # 用于记录每个类别的图像数量
    incomplete_categories = []  # 用于记录不满足12张图像的类别
    filtered_event_ids = []  # 用于存储过滤后的 event_id
    
    # 新建输出目录
    os.makedirs(output_dir, exist_ok=True)
    
    for i, event_id in enumerate(event_ids):
        if event_id not in image_event_mapping:
            continue
        category_with_prefix, img = image_event_mapping[event_id]
        
        # 检查每个类别的图像数量是否已达到限制
        if image_count[category_with_prefix] < category_limit:
            output_array.append(data_array[i])  # 保留该样本
            filtered_event_ids.append(event_id)  # 将符合条件的 event_id 添加到过滤列表
            image_count[category_with_prefix] += 1
            
            # 提取不带前缀的类别名
            # category = category_with_prefix.split("_", 1)[1]  # 提取不带前缀的类别名
            
            # 构建源路径（使用不带前缀的类别名）和目标路径
            # source_path = os.path.join(source_image_dir, "images", category, img)  # 使用不带前缀的类别名
            # category_output_dir = os.path.join(output_dir, category_with_prefix)  # 目标路径使用带前缀的类别名
            # os.makedirs(category_output_dir, exist_ok=True)
            # target_path = os.path.join(category_output_dir, img)
            
            # 拷贝文件
            # shutil.copyfile(source_path, target_path)
    
    # 检查是否存在不满足12张图像的类别
    for category, count in image_count.items():
        if count < category_limit:
            incomplete_categories.append((category, count))
    
    return incomplete_categories, filtered_event_ids

# 目录设置
csv_img_file_path = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Metadata/Image-specific/image_paths.csv"
base_fif_dir = "/mnt/dataset0/ldy/datasets/meg_dataset/original_preprocessed/preprocessed"
source_image_dir = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Images/"
meg_image_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/images_set"
output_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/Preprocessed"
filtered_image_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/images_set_filter"
subjects = [
    ('sub-02', 'preprocessed_P2-epo.fif'),
]

category_prefix_mapping = build_category_prefix_mapping(meg_image_dir)
full_image_event_mapping = get_full_image_event_mapping(csv_img_file_path, category_prefix_mapping)

image_event_mapping_train = {}
image_event_mapping_test = {}

for event_id, (category_with_prefix, img) in full_image_event_mapping.items():
    train_path = os.path.join(meg_image_dir, "training_images", category_with_prefix, img)
    test_path = os.path.join(meg_image_dir, "test_images", category_with_prefix, img)
    if os.path.exists(train_path):
        image_event_mapping_train[event_id] = (category_with_prefix, img)
    elif os.path.exists(test_path):
        image_event_mapping_test[event_id] = (category_with_prefix, img)

# 主处理代码
filtered_train_event_ids = []
filtered_test_event_ids = []

for subject_id, fif_filename in subjects:
    fif_file = os.path.join(base_fif_dir, fif_filename)
    data_array, event_ids, ch_names, times = load_and_crop_epochs(fif_file)
    print("event_ids", event_ids)
    train_array = []
    test_array = []
    
    # 过滤训练集和测试集，保留每个类别的前12张图像并获取过滤后的 event_ids
    incomplete_train_categories, filtered_train_event_ids = filter_and_copy_images(
        image_event_mapping_train, data_array, event_ids, train_array, os.path.join(filtered_image_dir, "training_images"))
    incomplete_test_categories, filtered_test_event_ids = filter_and_copy_images(
        image_event_mapping_test, data_array, event_ids, test_array, os.path.join(filtered_image_dir, "test_images"))

    # 将结果转换为 numpy 数组
    train_array = np.array(train_array)
    test_array = np.array(test_array)
    
    subject_output_dir = os.path.join(output_dir, subject_id)
    os.makedirs(subject_output_dir, exist_ok=True)
    
    save_data({'meg_data': train_array, 'ch_names': ch_names, 'times': times},
              os.path.join(subject_output_dir, "preprocessed_meg_training.pkl"))
    save_data({'meg_data': test_array, 'ch_names': ch_names, 'times': times},
              os.path.join(subject_output_dir, "preprocessed_meg_test.pkl"))

    # 输出不满足12张图像的类别
    if incomplete_train_categories:
        print(f"\n训练集中不满足12张图像的类别（数量不足） for {subject_id}:")
        for category, count in incomplete_train_categories:
            print(f"  类别 '{category}': 仅 {count} 张图像")
    
    if incomplete_test_categories:
        print(f"\n测试集中不满足12张图像的类别（数量不足） for {subject_id}:")
        for category, count in incomplete_test_categories:
            print(f"  类别 '{category}': 仅 {count} 张图像")

# 处理完成后，直接使用 filtered_train_event_ids 和 filtered_test_event_ids
print("处理完成！")
print("过滤后的训练集的 event_ids:", filtered_train_event_ids)
print("过滤后的测试集的 event_ids:", filtered_test_event_ids)

In [ ]:
import os
import pandas as pd
import os
import pickle
import numpy as np
import mne
import pandas as pd
from collections import defaultdict
import shutil

def load_and_crop_epochs(fif_file):
    epochs = mne.read_epochs(fif_file, preload=True)
    epochs.crop(tmin=0, tmax=1.0)
    data_array = epochs.get_data()
    event_ids = epochs.events[:, 2]
    
    # 获取非 999999 的索引
    valid_indices = np.where(event_ids != 999999)[0]
    data_array = data_array[valid_indices]
    event_ids = event_ids[valid_indices]
    
    # 获取唯一的 event_ids 的索引
    unique_indices = np.unique(event_ids, return_index=True)[1]
    data_array = data_array[unique_indices]
    event_ids = event_ids[unique_indices]
    
    return data_array, event_ids, epochs.ch_names, epochs.times

# 读取 meg 类别标签文件
csv_file_path = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Metadata/Concept-specific/image_concept_index.csv"
concept_df = pd.read_csv(csv_file_path, header=None, names=['Category_Label'])

# 检查行号是否与 event_id 一致
# 如果每行对应的行号就是 event_id，则行号即是 event_id
concept_df['Event_ID'] = concept_df.index + 1  # 添加 Event_ID 列

def count_samples_per_category(event_ids, concept_df):
    # 筛选出在过滤后的 event_ids 中的类别标签
    filtered_concept_df = concept_df[concept_df['Event_ID'].isin(event_ids)]

    # 统计每个类别在过滤后的数据中的样本数量
    category_counts = filtered_concept_df['Category_Label'].value_counts().sort_index()

    # 将统计结果转为字典并返回
    return category_counts.to_dict()

subjects = [
    ('sub-02', 'preprocessed_P2-epo.fif'),
]
base_fif_dir = "/mnt/dataset0/ldy/datasets/meg_dataset/original_preprocessed/preprocessed"

for subject_id, fif_filename in subjects:
    fif_file = os.path.join(base_fif_dir, fif_filename)
    data_array, event_ids, ch_names, times = load_and_crop_epochs(fif_file)

    # # 调用函数进行统计
    # category_counts = count_samples_per_category(event_ids, concept_df)

    # # 输出结果
    # print("每个类别在过滤后的 MEG 数据中样本数量:")
    # for category, count in category_counts.items():
    #     print(f"类别 {category}: {count} 个样本")


In [ ]:
import os
import pandas as pd
import os
import pickle
import numpy as np
import mne
import pandas as pd
from collections import defaultdict
import shutil

# 读取 meg 类别标签文件
csv_file_path = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Metadata/Concept-specific/image_concept_index.csv"
concept_df = pd.read_csv(csv_file_path, header=None, names=['Category_Label'])
# 检查行号是否与 event_id 一致
# 如果每行对应的行号就是 event_id，则行号即是 event_id
concept_df['Event_ID'] = concept_df.index + 1  # 添加 Event_ID 列


def count_samples_per_category(event_ids, concept_df):
    # 筛选出在过滤后的 event_ids 中的类别标签
    filtered_concept_df = concept_df[concept_df['Event_ID'].isin(event_ids)]

    # 统计每个类别在过滤后的数据中的样本数量
    category_counts = filtered_concept_df['Category_Label'].value_counts().sort_index()

    # 将统计结果转为字典并返回
    return category_counts.to_dict()
def load_event_ids_from_txt(filename):
    """
    从 txt 文件中加载 event_ids，返回一个整数列表。
    """
    with open(filename, 'r') as f:
        event_ids = [int(line.strip()) for line in f]
    return event_ids

train_img_event_ids = load_event_ids_from_txt("train_img_event_ids.txt")
test_img_event_ids = load_event_ids_from_txt("test_img_event_ids.txt")
# 调用函数进行统计
category_counts = count_samples_per_category(test_img_event_ids, concept_df)

# 输出结果
print("每个类别在过滤后的 MEG 数据中样本数量:")
for category, count in category_counts.items():
    print(f"类别 {category}: {count} 个样本")

In [ ]:
# 调用函数进行统计
category_counts = count_samples_per_category(train_img_event_ids, concept_df)

# 输出结果
print("每个类别在过滤后的 MEG 数据中样本数量:")
for category, count in category_counts.items():
    print(f"类别 {category}: {count} 个样本")

In [ ]:
import os
import pandas as pd


# 将 category_prefix_mapping 字典转换为 DataFrame，便于查看
category_prefix_mapping_df = pd.DataFrame(list(category_prefix_mapping.items()), columns=['Category', 'Category_Prefix'])

# 保存为 CSV 文件
category_mapping_file_path = "category_prefix_mapping.csv"
category_prefix_mapping_df.to_csv(category_mapping_file_path, index=False)

print(f"文件已保存到: {category_mapping_file_path}")


In [14]:
# save_data({'meg_data': train_array, 'ch_names': ch_names, 'times': times},
#             os.path.join(subject_output_dir, "preprocessed_meg_training.pkl"))
# save_data({'meg_data': test_array, 'ch_names': ch_names, 'times': times},
#             os.path.join(subject_output_dir, "preprocessed_meg_test.pkl"))


In [ ]:
# 保存过滤后的训练集和测试集 event_ids 到 txt 文件
def save_event_ids_to_txt(event_ids, filename):
    with open(filename, 'w') as f:
        for event_id in event_ids:
            f.write(f"{event_id}\n")

# 将 filtered_train_event_ids 和 filtered_test_event_ids 保存到文件
save_event_ids_to_txt(filtered_train_event_ids, "filtered_train_event_ids.txt")
save_event_ids_to_txt(filtered_test_event_ids, "filtered_test_event_ids.txt")

print("过滤后的训练集的 event_ids 已保存到 filtered_train_event_ids.txt")
print("过滤后的测试集的 event_ids 已保存到 filtered_test_event_ids.txt")


In [ ]:
import os
import pandas as pd

# CSV 文件路径
csv_img_file_path = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Metadata/Image-specific/image_paths.csv"
filtered_image_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/images_set_filter"

def load_img_event_id_mapping(csv_img_file_path):
    """
    从 CSV 文件中加载图像名称到 img_event_id 的映射。
    返回字典：{image_name: img_event_id}
    """
    df = pd.read_csv(csv_img_file_path, header=None)
    img_event_id_mapping = {os.path.basename(row[0]): idx + 1 for idx, row in df.iterrows()}  # 提取文件名作为键
    return img_event_id_mapping

def get_img_event_ids(image_dir, img_event_id_mapping):
    """
    根据图像目录中的文件名查找 img_event_id。
    返回 img_event_ids 列表。
    """
    img_event_ids = []
    for category_dir in sorted(os.listdir(image_dir)):  # 遍历类别目录
        category_path = os.path.join(image_dir, category_dir)
        if not os.path.isdir(category_path):
            continue
        for img_file in sorted(os.listdir(category_path)):  # 遍历图像文件
            img_event_id = img_event_id_mapping.get(img_file)
            if img_event_id is not None:
                img_event_ids.append(img_event_id)
    return img_event_ids

# 加载 img_event_id 映射
img_event_id_mapping = load_img_event_id_mapping(csv_img_file_path)

# 获取训练集和测试集的 img_event_ids
train_img_event_ids = get_img_event_ids(os.path.join(filtered_image_dir, "training_images"), img_event_id_mapping)
test_img_event_ids = get_img_event_ids(os.path.join(filtered_image_dir, "test_images"), img_event_id_mapping)

# 输出结果
print("训练集的 img_event_ids:", train_img_event_ids)
print("测试集的 img_event_ids:", test_img_event_ids)


In [ ]:
# 保存 img_event_ids 到 txt 文件
def save_img_event_ids_to_txt(event_ids, filename):
    with open(filename, 'w') as f:
        for event_id in event_ids:
            f.write(f"{event_id}\n")

# 保存训练集和测试集的 img_event_ids 到文件
save_img_event_ids_to_txt(train_img_event_ids, "train_img_event_ids.txt")
save_img_event_ids_to_txt(test_img_event_ids, "test_img_event_ids.txt")

print("训练集的 img_event_ids 已保存到 train_img_event_ids.txt")
print("测试集的 img_event_ids 已保存到 test_img_event_ids.txt")


In [ ]:
def load_event_ids_from_txt(filename):
    """
    从 txt 文件中加载 event_ids，返回一个整数列表。
    """
    with open(filename, 'r') as f:
        event_ids = [int(line.strip()) for line in f]
    return event_ids

def compare_event_ids(file1, file2):
    """
    对比两个文件中的 event_ids，返回差异。
    """
    event_ids_1 = set(load_event_ids_from_txt(file1))
    event_ids_2 = set(load_event_ids_from_txt(file2))
    
    # 找出只在 file1 中存在的 event_ids
    only_in_file1 = event_ids_1 - event_ids_2
    # 找出只在 file2 中存在的 event_ids
    only_in_file2 = event_ids_2 - event_ids_1
    
    return only_in_file1, only_in_file2

# 文件路径
filtered_test_event_ids_path = "filtered_test_event_ids.txt"
test_img_event_ids_path = "test_img_event_ids.txt"
filtered_train_event_ids_path = "filtered_train_event_ids.txt"
train_img_event_ids_path = "train_img_event_ids.txt"

# 对比测试集的差异
test_only_in_filtered, test_only_in_img = compare_event_ids(filtered_test_event_ids_path, test_img_event_ids_path)
print("只在 filtered_test_event_ids.txt 中的 event_ids:", test_only_in_filtered)
print("只在 test_img_event_ids.txt 中的 event_ids:", test_only_in_img)

# 对比训练集的差异
train_only_in_filtered, train_only_in_img = compare_event_ids(filtered_train_event_ids_path, train_img_event_ids_path)
print("只在 filtered_train_event_ids.txt 中的 event_ids:", train_only_in_filtered)
print("只在 train_img_event_ids.txt 中的 event_ids:", train_only_in_img)


In [ ]:
import os
import pickle
import numpy as np
import mne
import pandas as pd
from collections import defaultdict
import shutil

base_fif_dir = "/mnt/dataset0/ldy/datasets/meg_dataset/original_preprocessed/preprocessed"

def load_and_crop_epochs(fif_file):
    epochs = mne.read_epochs(fif_file, preload=True)
    epochs.crop(tmin=0, tmax=1.0)
    data_array = epochs.get_data()
    event_ids = epochs.events[:, 2]
    
    # 获取非 999999 的索引
    valid_indices = np.where(event_ids != 999999)[0]
    data_array = data_array[valid_indices]
    event_ids = event_ids[valid_indices]
    
    # 获取唯一的 event_ids 的索引
    unique_indices = np.unique(event_ids, return_index=True)[1]
    data_array = data_array[unique_indices]
    event_ids = event_ids[unique_indices]
    
    return data_array, event_ids, epochs.ch_names, epochs.times


subjects = [
    ('sub-02', 'preprocessed_P2-epo.fif'),
]
            



In [ ]:
# image_event_mapping_train

# 将字典转换为 DataFrame，便于查看
image_event_mapping_train_df = pd.DataFrame.from_dict(image_event_mapping_train, orient='index', columns=['Category_Prefix', 'Image'])

# 保存为 CSV 文件
train_file_path = "image_event_mapping_train.csv"
image_event_mapping_train_df.to_csv(train_file_path, index_label="Event_ID")

print(f"文件已保存到: {train_file_path}")



# image_event_mapping_test
# 将字典转换为 DataFrame，便于查看
image_event_mapping_test_df = pd.DataFrame.from_dict(image_event_mapping_test, orient='index', columns=['Category_Prefix', 'Image'])

# 保存为 CSV 文件
file_path = "image_event_mapping_test.csv"
image_event_mapping_test_df.to_csv(file_path, index_label="Event_ID")

print(f"文件已保存到: {file_path}")



In [ ]:
full_image_event_mapping

# full_image_event_mapping
# 将字典转换为 DataFrame，便于查看
full_image_event_mapping_df = pd.DataFrame.from_dict(full_image_event_mapping, orient='index', columns=['Category_Prefix', 'Image'])

# 保存为 CSV 文件
file_path = "full_image_event_mapping.csv"
full_image_event_mapping_df.to_csv(file_path, index_label="Event_ID")

print(f"文件已保存到: {file_path}")


In [ ]:
for subject_id, fif_filename in subjects:
    fif_file = os.path.join(base_fif_dir, fif_filename)
    data_array, event_ids, ch_names, times = load_and_crop_epochs(fif_file)
    
    train_array = []
    test_array = []
    
    for i, event_id in enumerate(event_ids):
        if event_id in image_event_mapping_train:
            train_array.append(data_array[i])
        elif event_id in image_event_mapping_test:
            test_array.append(data_array[i])

    train_array = np.array(train_array)
    test_array = np.array(test_array)
    
    subject_output_dir = os.path.join(output_dir, subject_id)
    os.makedirs(subject_output_dir, exist_ok=True)
    
    unmatched_train_keys = find_unmatched_keys(event_ids, image_event_mapping_train)
    unmatched_test_keys = find_unmatched_keys(event_ids, image_event_mapping_test)
    
    filled_train_data, filled_train_ids, train_insert_indices = insert_unmatched_events(
        unmatched_train_keys, image_event_mapping_train, data_array, event_ids)
    filled_test_data, filled_test_ids, test_insert_indices = insert_unmatched_events(
        unmatched_test_keys, image_event_mapping_test, data_array, event_ids)
    
    # Sort indices in descending order to avoid index shift during insertion
    train_insert_indices = sorted(train_insert_indices, reverse=True)
    test_insert_indices = sorted(test_insert_indices, reverse=True)

    # Insert filled data from the end of the array, ensuring indices stay consistent
    if filled_train_data.size > 0:
        for data, idx in zip(filled_train_data, train_insert_indices):
            # print("idx", idx)
            train_array = np.insert(train_array, idx, data, axis=0)

    if filled_test_data.size > 0:
        for data, idx in zip(filled_test_data, test_insert_indices):
            print("idx", idx)
            test_array = np.insert(test_array, idx, data, axis=0)

    # save_data({'meg_data': train_array, 'ch_names': ch_names, 'times': times},
    #           os.path.join(subject_output_dir, "preprocessed_meg_training.pkl"))
    # save_data({'meg_data': test_array, 'ch_names': ch_names, 'times': times},
    #           os.path.join(subject_output_dir, "preprocessed_meg_test.pkl"))

    print(f"\nUnmatched keys in training set for {subject_id}:")
    print(unmatched_train_keys)
    
    print(f"\nUnmatched keys in test set for {subject_id}:")
    print(unmatched_test_keys)

print("处理完成！")

In [ ]:
test_array

In [ ]:
import os
import pickle
import numpy as np
import mne
from tqdm import tqdm
from collections import defaultdict

# 目录设置
base_fif_dir = "/mnt/dataset0/ldy/datasets/meg_dataset/original_preprocessed/preprocessed"
source_image_dir = "/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Images/images"
meg_image_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/images_set"
output_dir = "/mnt/dataset0/ldy/datasets/THINGS_MEG/Preprocessed"
subjects = [
    ('sub-01', 'preprocessed_P1-epo.fif'),
    ('sub-02', 'preprocessed_P2-epo.fif'),
    ('sub-03', 'preprocessed_P3-epo.fif'),
    ('sub-04', 'preprocessed_P4-epo.fif'),
]

def save_data(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

def load_and_crop_epochs(fif_file):
    # 读取并裁剪MEG数据
    epochs = mne.read_epochs(fif_file, preload=True)
    epochs.crop(tmin=0, tmax=1.0)
    data_array = epochs.get_data()
    event_ids = epochs.events[:, 2]
    unique_indices = np.unique(event_ids[event_ids != 999999], return_index=True)[1]
    data_array = data_array[unique_indices]
    event_ids = event_ids[event_ids != 999999][unique_indices]
    return data_array, event_ids, epochs.ch_names, epochs.times

def build_category_prefix_mapping(meg_image_dir):
    # 从 training_images 和 test_images 目录中提取带编号的前缀和类别名映射
    category_prefix_mapping = {}
    
    for split in ["training_images", "test_images"]:
        split_dir = os.path.join(meg_image_dir, split)
        if os.path.isdir(split_dir):
            for category_with_prefix in os.listdir(split_dir):
                prefix, category = category_with_prefix.split("_", 1)
                category_prefix_mapping[category] = category_with_prefix  # 建立类别到带前缀名称的映射
                
    return category_prefix_mapping

def get_full_image_event_mapping(base_dir, category_prefix_mapping):
    # 获取完整的图像事件的映射，确保事件id从0到22447连续
    event_id = 1
    full_image_event_mapping = {}
    categories = sorted(os.listdir(base_dir))
    
    for category in categories:
        category_path = os.path.join(base_dir, category)
        
        # 确认该类别在带编号前缀的映射中
        if os.path.isdir(category_path) and category in category_prefix_mapping:
            category_with_prefix = category_prefix_mapping[category]
            images = sorted(os.listdir(category_path))
            for img in images:
                full_image_event_mapping[event_id] = (category_with_prefix, img)
                event_id += 1
                
    return full_image_event_mapping

category_prefix_mapping = build_category_prefix_mapping(meg_image_dir)
full_image_event_mapping = get_full_image_event_mapping(source_image_dir, category_prefix_mapping)

train_images_expected = set()
test_images_expected = set()

for category_with_prefix in category_prefix_mapping.values():
    train_dir = os.path.join(meg_image_dir, "training_images", category_with_prefix)
    test_dir = os.path.join(meg_image_dir, "test_images", category_with_prefix)

    if os.path.exists(train_dir):
        train_images_expected.update(
            [f"{category_with_prefix}/{img}" for img in sorted(os.listdir(train_dir))]
        )
    if os.path.exists(test_dir):
        test_images_expected.update(
            [f"{category_with_prefix}/{img}" for img in sorted(os.listdir(test_dir))]
        )

image_event_mapping_train = {}
image_event_mapping_test = {}

for event_id, (category_with_prefix, img) in full_image_event_mapping.items():
    train_path = os.path.join(meg_image_dir, "training_images", category_with_prefix, img)
    test_path = os.path.join(meg_image_dir, "test_images", category_with_prefix, img)
    
    if os.path.exists(train_path):
        image_event_mapping_train[event_id] = (category_with_prefix, img)
    elif os.path.exists(test_path):
        image_event_mapping_test[event_id] = (category_with_prefix, img)

# 统计哪些键不在加载的 event_ids 中
def find_unmatched_keys(event_ids, image_event_mapping):
    return [key for key in image_event_mapping.keys() if key not in event_ids]

# 加载和保存每个受试者的MEG数据
for subject_id, fif_filename in subjects:
    fif_file = os.path.join(base_fif_dir, fif_filename)
    data_array, event_ids, ch_names, times = load_and_crop_epochs(fif_file)
    
    train_array = []
    test_array = []
    
    for i, event_id in enumerate(event_ids):
        if event_id in image_event_mapping_train:
            train_array.append(data_array[i])
        elif event_id in image_event_mapping_test:
            test_array.append(data_array[i])

    train_array = np.array(train_array)
    test_array = np.array(test_array)
    
    subject_output_dir = os.path.join(output_dir, subject_id)
    os.makedirs(subject_output_dir, exist_ok=True)
    
    save_data({'meg_data': train_array, 'ch_names': ch_names, 'times': times},
              os.path.join(subject_output_dir, "preprocessed_meg_training.pkl"))
    save_data({'meg_data': test_array, 'ch_names': ch_names, 'times': times},
              os.path.join(subject_output_dir, "preprocessed_meg_test.pkl"))

    # 输出未匹配的 event_id
    unmatched_train_keys = find_unmatched_keys(event_ids, image_event_mapping_train)
    unmatched_test_keys = find_unmatched_keys(event_ids, image_event_mapping_test)

    print(f"\nUnmatched keys in training set for {subject_id}:")
    print(unmatched_train_keys)
    
    print(f"\nUnmatched keys in test set for {subject_id}:")
    print(unmatched_test_keys)

    # 删除 unmatched_train_keys 和 unmatched_test_keys 对应的原图
    for unmatched_keys, mapping, split_name in [
        (unmatched_train_keys, image_event_mapping_train, "training_images"),
        (unmatched_test_keys, image_event_mapping_test, "test_images"),
    ]:
        for event_id in unmatched_keys:
            if event_id in mapping:
                category, img_name = mapping[event_id]
                img_path = os.path.join(meg_image_dir, split_name, category, img_name)
                if os.path.exists(img_path):
                    os.remove(img_path)
                    print(f"Deleted image: {img_path}")
                else:
                    print(f"Image not found for deletion: {img_path}")

print("处理完成！")
